In [ ]:
import requests

api_url = 'https://api.myanimelist.net/v2'

# A Client ID is needed (https://myanimelist.net/apiconfig)
#with open('client_id.txt', 'r') as f:
#    CLIENT_ID = f.read()

headers = {'X-MAL-CLIENT-ID': "ec1c4b7aa5f59e854390b2259667340c"}

def get_data(endpoint, params=None):
    url = api_url + endpoint
    if params:
        url += '?' + '&'.join(f'{key}={value}' for key, value in params.items())
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()


In [ ]:
import json
import os

scraping_save_pages = '../My Anime List Scrapping/data/data_tmp/recomm_pages'

if not os.path.exists(scraping_save_pages): # Create saving directory if it doesn't exist
    os.makedirs(scraping_save_pages)
endpoint_ranking = f'/anime/ranking'
limit = 500
anime_keys = ['id', 'title']

def scrape_page(page):
    params = {'ranking_type': 'bypopularity', 'limit': limit, 'offset': page*limit, 'fields': ','.join(anime_keys)}
    data = get_data(endpoint_ranking, params)
    useful = [anime['node'] for anime in data['data']]
    with open(scraping_save_pages + f'/page{str(page).zfill(2)}.json', 'w') as f:
        json.dump(useful, f, indent=4)
        


In [ ]:
import math

# 22 April 2023
previous_total_anime = 50_900
previous_last_page = math.ceil(previous_total_anime / limit) - 1

data = get_data(endpoint_ranking, {'ranking_type': 'bypopularity', 'limit': limit, 'offset': previous_last_page*limit, 'fields': ','.join(anime_keys)})
#data = get_data(endpoint,{'ranking_type': 'bypopularity', 'limit': limit, 'offset': previous_last_page*limit, 'fields': ','.join(str(v) for v in 'recommendations')})
#assert len(data['data']) > 0
assert 'next' not in data['paging']


last_page = previous_last_page

last_page

In [ ]:
import json
import os
import requests

scraping_save_pages = '../My Anime List Scrapping/data/data_tmp/recomm_pages'

if not os.path.exists(scraping_save_pages): # Create saving directory if it doesn't exist
    os.makedirs(scraping_save_pages)

api_url = 'https://api.myanimelist.net/v2'
headers = {'X-MAL-CLIENT-ID': "ec1c4b7aa5f59e854390b2259667340c"}
endpoint = f'/anime/'
limit = 500
anime_keys = ['id', 'title','recommendations']
scraping_save_pages = '../My Anime List Scrapping/data/data_tmp/recomm_pages'
ecount=0
def scrape_page(page):
    params = { 'fields': ','.join(anime_keys)}
    for anime_id in range(1, 100000):
        try:
            data = get_recommendations(endpoint,anime_id,params)
            useful = [anime['node'] for anime in data['data']]
        except requests.exceptions.HTTPError as e:
            ecount+=1
    print(ecount + 'errors not exisitng IDs')
    with open(scraping_save_pages + f'/page{str(page).zfill(2)}.json', 'w') as f:
        json.dump(useful, f, indent=4)
def get_recommendations(endpoint,anime_id, params=None):
    url=api_url + endpoint
    if params:
        url += f'anime_id' + '?' + '&'.join(f'{key}={value}' for key, value in params.items())
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()
        


In [ ]:
import math

# 22 April 2023
previous_total_anime = 50_900
previous_last_page = math.ceil(previous_total_anime / limit) - 1
for anime_id in range(100, 100000):
        try:
            data = get_recommendations(endpoint,anime_id,{'fields': ','.join(anime_keys)})
        except requests.exceptions.HTTPError as e:
            ecount+=1
            print(ecount)
print(ecount + 'errors not exisitng IDs')
assert 'next' not in data['paging']
last_page = previous_last_page

last_page

In [ ]:
import tqdm
import time

for page in tqdm.trange(last_page+1):
    scrape_page(page)
    time.sleep(1)

In [ ]:
import os

scraping_save_pages = 'data/data_tmp/recomm_pages'

data = []
for file_name in os.listdir(scraping_save_pages):
    file_path = os.path.join(scraping_save_pages, file_name)
    with open(file_path, 'r') as f:
        file = json.load(f)
    data.extend(file)

len(data),type(data)

In [ ]:
len(data),len(data[1])
data[1]

In [ ]:
with open('data/data_tmp/recomm_raw.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
import pandas as pd
import numpy as np
import json

with open('data/data_tmp/recomm_raw.json', 'r') as f:
    data = json.load(f)
anime = pd.json_normalize(data, sep='_')

In [ ]:
anime.columns

In [ ]:
anime.sample(10)